In [2]:
from preprocess import process_data
import numpy as np

In [3]:
sub_df, train_matrix, holdout_list = process_data()

In [4]:
sub_df

,J5,J7,J8,J13,J15,J16,J17,J18,J19,J20
user_0,-7.52,-9.85,4.17,-7.18,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85
user_1,-2.38,-0.73,-5.34,4.42,4.56,-0.97,4.66,-0.68,3.30,-1.21
user_2,9.03,9.03,9.27,9.37,-6.36,-6.89,-7.86,9.03,9.03,9.03
user_3,1.80,-2.82,6.21,6.31,-7.23,-6.65,1.17,-6.60,-3.64,-2.09
user_4,1.36,7.04,4.61,-3.93,-2.33,-9.66,2.72,-1.36,2.57,4.51
...,...,...,...,...,...,...,...,...,...,...
user_24978,3.20,-8.79,-0.53,-7.14,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46
user_24979,0.87,-3.50,5.78,-7.48,-1.75,6.60,3.54,1.50,7.67,-0.44
user_24980,-7.77,6.70,-6.75,-6.46,-6.80,-6.41,-6.99,7.23,6.75,-6.99
user_24981,-9.71,4.56,-8.30,-9.47,3.45,-0.92,-4.51,-4.13,-5.73,-9.51


In [5]:
train_matrix

,J5,J7,J8,J13,J15,J16,J17,J18,J19,J20
user_0,NaN,NaN,4.17,-7.18,-7.18,NaN,-7.43,-9.81,NaN,-9.85
user_1,NaN,NaN,-5.34,4.42,4.56,NaN,4.66,-0.68,NaN,-1.21
user_2,NaN,9.03,NaN,9.37,-6.36,-6.89,NaN,9.03,9.03,NaN
user_3,1.80,NaN,NaN,6.31,-7.23,-6.65,NaN,NaN,-3.64,-2.09
user_4,1.36,NaN,4.61,-3.93,NaN,NaN,2.72,-1.36,NaN,4.51
...,...,...,...,...,...,...,...,...,...,...
user_24978,NaN,-8.79,-0.53,NaN,-3.50,NaN,-3.98,-9.22,NaN,-6.46
user_24979,0.87,-3.50,5.78,NaN,NaN,NaN,3.54,NaN,7.67,-0.44
user_24980,NaN,6.70,NaN,-6.46,-6.80,-6.41,-6.99,NaN,6.75,NaN
user_24981,NaN,4.56,-8.30,NaN,NaN,-0.92,NaN,-4.13,-5.73,-9.51


In [11]:
len(holdout_list)/4

[('user_0', np.str_('J19'), np.float64(-9.85)),
 ('user_0', np.str_('J7'), np.float64(-9.85)),
 ('user_0', np.str_('J16'), np.float64(-7.52)),
 ('user_0', np.str_('J5'), np.float64(-7.52)),
 ('user_1', np.str_('J5'), np.float64(-2.38)),
 ('user_1', np.str_('J7'), np.float64(-0.73)),
 ('user_1', np.str_('J19'), np.float64(3.3)),
 ('user_1', np.str_('J16'), np.float64(-0.97)),
 ('user_2', np.str_('J20'), np.float64(9.03)),
 ('user_2', np.str_('J8'), np.float64(9.27)),
 ('user_2', np.str_('J5'), np.float64(9.03)),
 ('user_2', np.str_('J17'), np.float64(-7.86)),
 ('user_3', np.str_('J7'), np.float64(-2.82)),
 ('user_3', np.str_('J18'), np.float64(-6.6)),
 ('user_3', np.str_('J17'), np.float64(1.17)),
 ('user_3', np.str_('J8'), np.float64(6.21)),
 ('user_4', np.str_('J7'), np.float64(7.04)),
 ('user_4', np.str_('J16'), np.float64(-9.66)),
 ('user_4', np.str_('J15'), np.float64(-2.33)),
 ('user_4', np.str_('J19'), np.float64(2.57)),
 ('user_5', np.str_('J19'), np.float64(-2.28)),
 ('user_5',

In [9]:
train_matrix.shape

(24983, 10)

In [ ]:
def factorization(learning_rate, regularization, num_epochs, R_missing, feature_num, output):

    """ Perform the SGD factorization with the given parameters"""
    
    N,M = R_missing.shape
    
    U_factorized = np.random.normal(0, 0.01,(N,feature_num))
    V_factorized = np.random.normal(0,0.01,(M,feature_num)) # initialze the U and V matrices

    for epoch in range(num_epochs):

        for i in range(N):
            for j in range(M):
                
                # Only consider the observed values
                if not np.isnan(R_missing[i,j]):

                    prediction = np.dot(U_factorized[i], V_factorized[j])
                    error = R_missing[i,j] - prediction

                    # Update the U and V matrices
                    U_factorized[i] += learning_rate * (error * V_factorized[j] - regularization * U_factorized[i])
                    V_factorized[j] += learning_rate * (error * U_factorized[i] - regularization * V_factorized[j]) 
                    
        if output:
            if epoch % 10 == 0:
            
                # Calculatre the total loss on observed entrices
                observed_indices = ~np.isnan(R_missing)
                loss = np.sum((R_missing[observed_indices] - (U_factorized @ V_factorized.T)[observed_indices])**2)
                print("Epoch:", epoch, "Loss:", loss)
    
    return U_factorized, V_factorized